In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing.plugin_event_accumulator import EventAccumulator
import glob
import numpy as np

In [2]:
# income
input_date = '2022-07-06/17-43-22'
model_date = '2022-07-06/17-43-22'
data_name = 'income'
y_lim = [85.0, 86.5]
# # mnist
# input_date = '2022-07-06/17-43-24'
# model_date = '2022-07-06/17-43-23'
# data_name = 'mnist'
# y_lim = [95, 99]
# # blog
# input_date = '2022-07-07/00-51-31'
# model_date = '2022-07-07/00-50-56'
# data_name = 'blog'
# y_lim = [77, 84]
# # covertype
input_date = '2022-07-07/00-52-34'
model_date = '2022-07-07/00-52-03'
data_name = 'covertype'
y_lim = [71, 78]

num_comb = 32
num_path = 10

In [3]:
input_root = f'outputs/multirun/main_all/supervised/{data_name}/input/{input_date}/**/events*'
model_root = f'outputs/multirun/main_all/supervised/{data_name}/model/{model_date}/**/events*'

In [4]:
def get_results(root):
    path = glob.glob(root)
    path.sort()
    path = path[:num_path]
    results = [[] for _ in range(num_comb)]
    for p in path:
        ea = EventAccumulator(p)
        ea.Reload()
        # acc
        tag = ea.Tags()['tensors'][-1]
        for i, t in enumerate(ea.Tensors(tag)):
            results[i] += [t.tensor_proto.float_val[0]]
    return results

In [5]:
def show_box(results, title):
    # 箱ひげ図
    plt.figure(figsize=(100, 40))
    fig, ax = plt.subplots()

    bp = ax.boxplot(results)#, whis=np.inf)
    ax.set_xticklabels(list(range(num_comb)))

    m = np.mean(results[0])
    plt.plot([0, num_comb + 1], [m, m])

    plt.title(title)
    plt.xlabel('augment number')
    plt.ylabel('acc')
    # Y軸のメモリのrange
    plt.ylim(y_lim)
    plt.xlim([0, num_comb + 1])
    plt.grid()

    # 描画
    # plt.show()
    plt.savefig(f'outputs/results/{data_name}_{title}.png', format="png", dpi=100)
    plt.close()


In [6]:
results_input = get_results(input_root)
results_model = get_results(model_root)
print(len(results_input[0]), len(results_model[0]))
show_box(results_input, 'input')
show_box(results_model, 'model')

10 10


<Figure size 7200x2880 with 0 Axes>

<Figure size 7200x2880 with 0 Axes>

In [3]:
from data.data_loader import read_csv

In [5]:
df, _ = read_csv('dataset/blog/blogData_train.csv', 280, columns=None, missing_fn=None)

In [27]:
for i in range(277, 278):
    print(df[280].unique())

KeyError: 280